In [1]:
import pandas as pd
import numpy as np
import time
import datetime
import pickle
import random

In [2]:
import logging
LOG_FORMAT = "%(asctime)s - %(levelname)s - %(message)s"
# logging.basicConfig(level=logging.INFO, filename='log_build_dataset', format=LOG_FORMAT)
logging.basicConfig(level=logging.INFO, format=LOG_FORMAT)


In [3]:
SEED = 0

# Read

In [4]:
logging.info('start reading')

pickle_path = '../user_data/data_sort_out/train_dataset_mp.pickle'
with open(pickle_path, 'rb') as f:
    train_dataset_mp = pickle.load(f)
key_list_train, mp_action_train, mp_distance_train, mp_order_train = \
train_dataset_mp['key_list'], train_dataset_mp['mp_action'], train_dataset_mp['mp_distance'], train_dataset_mp['mp_order']

pickle_path = '../user_data/data_sort_out/testA_dataset_mp.pickle'
with open(pickle_path, 'rb') as f:
    testA_dataset_mp = pickle.load(f)
key_list_testA, mp_action_testA, mp_distance_testA, mp_order_testA = \
testA_dataset_mp['key_list'], testA_dataset_mp['mp_action'], testA_dataset_mp['mp_distance'], testA_dataset_mp['mp_order']

pickle_path = '../user_data/data_sort_out/testB_dataset_mp.pickle'
with open(pickle_path, 'rb') as f:
    testB_dataset_mp = pickle.load(f)
key_list_testB, mp_action_testB, mp_distance_testB, mp_order_testB, mp_distance_testB_detail = \
testB_dataset_mp['key_list'], testB_dataset_mp['mp_action'], testB_dataset_mp['mp_distance'], testB_dataset_mp['mp_order'], testB_dataset_mp['mp_distance_detail']
logging.info('finish reading')




1175.2078161239624

key_list_train, mp_action_train, mp_distance_train, mp_order_train

key_list_testA, mp_action_testA, mp_distance_testA, mp_order_testA

df_actions_train, df_couriers_train, df_distances_train, df_orders_train

df_actions_testA, df_couriers_testA, df_distances_testA, df_orders_testA


# Test

## testA

In [5]:
from IPython.display import display, HTML
def show_df(df):
    display(HTML(df.to_html()))

In [6]:
logging.info('start')
testA_know_lens, testA_lens, testA_impossible_idxs = [], [], []
for (date, courier, wave) in key_list_testA:
    df_action = mp_action_testA[date][courier][wave]
    testA_lens.append(df_action.shape[0])
    df_konw = df_action.query('expect_time != 0')
    df_unkonw = df_action.query('expect_time == 0')
    testA_know_lens.append(df_konw.shape[0])
    impossible_idx = []
    konw_tracking_id = set(df_konw['tracking_id'])
    for i, idx in enumerate(df_unkonw.index):
        if df_unkonw.at[idx, 'tracking_id'] not in konw_tracking_id and df_unkonw.at[idx, 'action_type'] == 'DELIVERY':
            impossible_idx.append(i)
    testA_impossible_idxs.append(impossible_idx)
    
logging.info('end')

2020-04-09 14:26:38,584 - INFO - start
2020-04-09 14:26:54,433 - INFO - end


In [7]:
df_testA_info = pd.DataFrame()
df_testA_info['know_lens'] = testA_know_lens
df_testA_info['lens'] = testA_lens
df_testA_info['impossible_idxs'] = testA_impossible_idxs
se_testA_lens_know_lens = df_testA_info.groupby('lens').apply(lambda x : x['know_lens'].values)
mp_test_lens_know_lens = se_testA_lens_know_lens.to_dict()
mp_test_lens_know_lens

{6: array([3, 3, 2, ..., 2, 3, 2]),
 8: array([4, 4, 3, 3, 3, 3, 4, 3, 4, 3, 2, 4, 2, 4, 5, 5, 2, 3, 3, 3, 5, 4,
        3, 5, 3, 5, 3, 3, 5, 5, 2, 4, 3, 3, 4, 2, 4, 4, 4, 5, 3, 2, 2, 2,
        5, 4, 5, 3, 3, 2, 5, 3, 2, 4, 5, 4, 3, 2, 4, 4, 3, 3, 4, 2, 2, 2,
        3, 4, 5, 5, 2, 5, 5, 4, 3, 5, 3, 3, 3, 3, 2, 5, 4, 3, 3, 3, 4, 2,
        2, 2, 4, 4, 5, 5, 5, 2, 4, 5, 5, 4, 3, 4, 2, 4, 3, 4, 4, 5, 3, 5,
        5, 5, 5, 3, 2, 4, 3, 4, 2, 2, 2, 4, 5, 3, 5, 3, 3, 4, 4, 5, 3, 3,
        5, 3, 3, 4, 2, 4, 4, 4, 4, 2, 4, 5, 3, 4, 2, 5, 5, 3, 5, 3, 4, 5,
        2, 2, 5, 2, 2, 4, 4, 4, 5, 3, 4, 5, 3, 3, 4, 4, 4, 3, 5, 5, 4, 5,
        3, 4, 5, 5, 2, 3, 4, 5, 3, 4, 5, 5, 4, 3, 3, 5, 3, 3, 5, 5, 2, 3,
        5, 3, 3, 3, 3, 2, 5, 2, 2, 3, 4, 4, 2, 3, 3, 3, 2, 3, 3, 2, 5, 2,
        2, 4, 4, 5, 2, 5, 5, 3, 3, 2, 4, 3, 2, 4, 2, 3, 4, 2, 5, 4, 2, 3,
        3, 3, 5, 2, 3, 2, 5, 4, 2, 4, 5, 3, 4, 5, 5, 4, 4, 5, 4, 5, 4, 4,
        3, 2, 3, 5, 4, 4, 3, 3, 4, 4, 3, 2, 4, 3, 2, 5, 4, 2, 4, 3, 3, 5,

## testB

In [5]:
logging.info('start')
testB_know_lens, testB_lens, testB_impossible_idxs = [], [], []
for (date, courier, wave) in key_list_testB:
    df_action = mp_action_testB[date][courier][wave]
    testB_lens.append(df_action.shape[0])
    df_konw = df_action.query('expect_time != 0')
    df_unkonw = df_action.query('expect_time == 0')
    testB_know_lens.append(df_konw.shape[0])
    impossible_idx = []
    konw_tracking_id = set(df_konw['tracking_id'])
    for i, idx in enumerate(df_unkonw.index):
        if df_unkonw.at[idx, 'tracking_id'] not in konw_tracking_id and df_unkonw.at[idx, 'action_type'] == 'DELIVERY':
            impossible_idx.append(i)
    testB_impossible_idxs.append(impossible_idx)
    
logging.info('end')

2020-04-13 21:27:54,378 - INFO - start
2020-04-13 21:28:14,006 - INFO - end


In [6]:
df_testB_info = pd.DataFrame()
df_testB_info['know_lens'] = testB_know_lens
df_testB_info['lens'] = testB_lens
df_testB_info['impossible_idxs'] = testB_impossible_idxs


# Train

In [35]:
logging.info('start train')
train_know_lens, train_lens, train_impossible_idxs = [], [], []
know_lens_list, list_impossible_dict = [], []
for i, (date, courier, wave) in enumerate(key_list_train):
    df_action = mp_action_train[date][courier][wave]
    
    train_len = df_action.shape[0]
    train_lens.append(train_len)
    
    train_know_len = None
    
    if train_len in mp_test_lens_know_lens:
        know_set = sorted(list(set(mp_test_lens_know_lens[train_len])))
        np.random.seed(i)        
        train_know_len = np.random.choice(know_set, size =1)[0]
    else:
        know_set = [2,]
        train_know_len = train_len / 2
    

    impossible_dict = {}
    for know_len in know_set:
        impossible_set = set()
        df_action_unknow = df_action.iloc[know_len: ]
        konw_tracking_id = set(df_action.iloc[: know_len].tracking_id)
        for i, idx in enumerate(df_action_unknow.index):
            if df_action_unknow.at[idx, 'tracking_id'] not in konw_tracking_id and\
            df_action_unknow.at[idx, 'action_type'] == 'DELIVERY':
                impossible_set.add(i)
        impossible_dict[know_len] = impossible_set
    
    know_lens_list.append(know_set)
    list_impossible_dict.append(impossible_dict)
    train_know_lens.append(int(train_know_len))
    
    train_know_len = int(train_know_len)
    
    df_konw = df_action.iloc[:train_know_len]
    df_unkonw = df_action.iloc[train_know_len:]
    
    impossible_idx = []
    konw_tracking_id = set(df_konw['tracking_id'])
    for i, idx in enumerate(df_unkonw.index):
        if df_unkonw.at[idx, 'tracking_id'] not in konw_tracking_id and df_unkonw.at[idx, 'action_type'] == 'DELIVERY':
            impossible_idx.append(i)
    train_impossible_idxs.append(impossible_idx)
logging.info('end train')


2020-04-09 15:29:46,428 - INFO - start train
2020-04-09 15:33:38,496 - INFO - end train


In [36]:
df_train_info = pd.DataFrame()
df_train_info['know_lens'] = train_know_lens
df_train_info['lens'] = train_lens
df_train_info['impossible_idxs'] = train_impossible_idxs
df_train_info['know_lens_list'] = know_lens_list
df_train_info['impossible_idxs_list'] = list_impossible_dict


# Pickle

In [10]:
import pickle
PICKLE_PATH = '../user_data/build_dataset'

In [9]:
with open('%s/df_train_info.pickle'%(PICKLE_PATH), 'wb') as f:
    pickle.dump(df_train_info, f)
with open('%s/df_testA_info.pickle'%(PICKLE_PATH), 'wb') as f:
    pickle.dump(df_testA_info, f)

NameError: name 'PICKLE_PATH' is not defined

In [39]:
with open('%s/key_list_train.pickle'%(PICKLE_PATH), 'wb') as f:
    pickle.dump(key_list_train, f)
with open('%s/key_list_testA.pickle'%(PICKLE_PATH), 'wb') as f:
    pickle.dump(key_list_testA, f)

In [11]:
with open('%s/df_testB_info.pickle'%(PICKLE_PATH), 'wb') as f:
    pickle.dump(df_testB_info, f)